In [2]:
# Cell 1 — Setup & helper functions
# Run first

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import os, time, joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.inspection import permutation_importance

# Optuna
try:
    import optuna
except Exception:
    !pip install -q optuna
    import optuna
from optuna.visualization.matplotlib import plot_optimization_history, plot_param_importances

# folders
os.makedirs("figures", exist_ok=True)
os.makedirs("models", exist_ok=True)
os.makedirs("optuna_studies", exist_ok=True)

# Helper to save matplotlib Figure or Axes
def save_plot_fig(fig_or_ax, filename):
    if hasattr(fig_or_ax, "figure"):  # it's an Axes
        fig = fig_or_ax.figure
    else:
        fig = fig_or_ax
    fig.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved: {filename}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.8 MB/s eta 0:00:00


In [4]:
# Cell 2 — Load & inspect dataset
# Make sure train_dataset.csv is in current working directory

df = pd.read_csv("train_dataset.csv")
print("Dataset shape:", df.shape)
display(df.head())

target_col = "smoking"   # as you provided
print(f"Target column: {target_col}")
print("Target value counts:\n", df[target_col].value_counts(dropna=False))
print("Any NaN in target:", df[target_col].isna().sum())

# Drop rows where target is NaN
df = df.dropna(subset=[target_col])
print("Shape after dropping NaN target rows:", df.shape)


Dataset shape: (27723, 23)


,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,35,170,85,97.0,0.9,0.9,1,1,118,78,...,70.0,142.0,19.8,1.0,1.0,61.0,115.0,125.0,1.0,1.0
1,20,175,110,110.0,0.7,0.9,1,1,119,79,...,71.0,114.0,15.9,1.0,1.1,19.0,25.0,30.0,1.0,0.0
2,45,155,65,86.0,0.9,0.9,1,1,110,80,...,57.0,112.0,13.7,3.0,0.6,1090.0,1400.0,276.0,0.0,0.0
3,45,165,80,94.0,0.8,0.7,1,1,158,88,...,46.0,91.0,16.9,1.0,0.9,32.0,36.0,36.0,0.0,0.0
4,20,165,60,81.0,1.5,0.1,1,1,109,64,...,47.0,92.0,14.9,1.0,1.2,26.0,28.0,15.0,0.0,0.0


Target column: smoking
Target value counts:
 smoking
0.0    17610
1.0    10112
NaN        1
Name: count, dtype: int64
Any NaN in target: 1
Shape after dropping NaN target rows: (27722, 23)


In [5]:
# Cell 3 — Preprocess: encode categorical features, split and scale
# This cell will create: X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled

# Separate features & label
X = df.drop(columns=[target_col]).copy()
y_raw = df[target_col].copy()

# One-hot encode object/categorical columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
if len(cat_cols) > 0:
    print("One-hot encoding categorical columns:", cat_cols)
    X = pd.get_dummies(X, columns=cat_cols, drop_first=True)
else:
    print("No categorical columns to one-hot encode.")

# Label encode target
le = LabelEncoder()
y = le.fit_transform(y_raw)
class_names = list(le.classes_)
n_classes = len(class_names)
print("Classes mapping:", dict(zip(class_names, range(n_classes))))

# Train-test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save encoder & scaler
joblib.dump(le, "models/label_encoder.joblib")
joblib.dump(scaler, "models/scaler.joblib")
print("Saved label encoder and scaler.")


No categorical columns to one-hot encode.
Classes mapping: {np.float64(0.0): 0, np.float64(1.0): 1}
Train shape: (22177, 22) Test shape: (5545, 22)
Saved label encoder and scaler.


In [8]:
# Cell 4 — Train baseline models and evaluate

def make_logreg_default(n_classes):
    if n_classes == 2:
        return LogisticRegression(solver="liblinear", max_iter=2000, random_state=42)
    else:
        return LogisticRegression(solver="lbfgs", multi_class="multinomial", max_iter=2000, random_state=42)

# Logistic Regression baseline
lr_base = make_logreg_default(n_classes)
print("Training Logistic Regression baseline...")
lr_base.fit(X_train_scaled, y_train)
pred_lr_base = lr_base.predict(X_test_scaled)
acc_lr_base = accuracy_score(y_test, pred_lr_base)
print("Logistic Regression Accuracy:", acc_lr_base)

# SVM baseline
print("Training SVM (RBF) baseline...")
svm_base = SVC(kernel="rbf", C=1.0, gamma="scale", probability=False, random_state=42)
svm_base.fit(X_train_scaled, y_train)
pred_svm_base = svm_base.predict(X_test_scaled)
acc_svm_base = accuracy_score(y_test, pred_svm_base)
print("SVM Accuracy:", acc_svm_base)

# MLP baseline
print("Training MLP baseline...")
mlp_base = MLPClassifier(hidden_layer_sizes=(128,64), activation='relu', solver='adam',
                         max_iter=200, early_stopping=True, random_state=42)
mlp_base.fit(X_train_scaled, y_train)
pred_mlp_base = mlp_base.predict(X_test_scaled)
acc_mlp_base = accuracy_score(y_test, pred_mlp_base)
print("Neural Network Accuracy:", acc_mlp_base)

# Save baseline models
joblib.dump(lr_base, "models/lr_base.joblib")
joblib.dump(svm_base, "models/svm_base.joblib")
joblib.dump(mlp_base, "models/mlp_base.joblib")
print("Saved baseline models under models/ folder.")


Training Logistic Regression baseline...
Logistic Regression Accuracy: 0.7128944995491434
Training SVM (RBF) baseline...
SVM Accuracy: 0.7428313796212804
Training MLP baseline...
Neural Network Accuracy: 0.745536519386835
Saved baseline models under models/ folder.


In [9]:
# Cell 5 — GridSearchCV for three models (small/moderate grids)
# NOTE: This may take time depending on data size. Adjust grids if slow.

results_grid = []

# Logistic Regression grid
print("GridSearch: Logistic Regression")
lr = make_logreg_default(n_classes)
if n_classes == 2:
    lr_param_grid = {"C":[0.01,0.1,1.0,10.0], "penalty":["l2"]}
else:
    lr_param_grid = {"C":[0.01,0.1,1.0,10.0], "penalty":["l2"], "solver":["lbfgs"]}

gs_lr = GridSearchCV(lr, lr_param_grid, cv=3, scoring="accuracy", n_jobs=-1, verbose=1)
gs_lr.fit(X_train_scaled, y_train)
best_lr = gs_lr.best_estimator_
pred_lr = best_lr.predict(X_test_scaled)
acc_lr = accuracy_score(y_test, pred_lr)
print("Best LR params:", gs_lr.best_params_)
print("Best LR CV accuracy:", gs_lr.best_score_)
print("Test accuracy (LR):", acc_lr)
joblib.dump(gs_lr, "models/gs_lr.joblib")
results_grid.append({"model":"LogisticRegression", "cv_best":gs_lr.best_score_, "test_acc":acc_lr, "best_params":gs_lr.best_params_})

# SVM grid
print("\nGridSearch: SVM (RBF)")
svm = SVC(kernel='rbf', probability=False, random_state=42)
svm_param_grid = {"C":[0.1,1.0,10.0], "gamma":["scale", 0.01, 0.001]}
gs_svm = GridSearchCV(svm, svm_param_grid, cv=3, scoring="accuracy", n_jobs=-1, verbose=1)
gs_svm.fit(X_train_scaled, y_train)
best_svm = gs_svm.best_estimator_
pred_svm = best_svm.predict(X_test_scaled)
acc_svm = accuracy_score(y_test, pred_svm)
print("Best SVM params:", gs_svm.best_params_)
print("Best SVM CV accuracy:", gs_svm.best_score_)
print("Test accuracy (SVM):", acc_svm)
joblib.dump(gs_svm, "models/gs_svm.joblib")
results_grid.append({"model":"SVM", "cv_best":gs_svm.best_score_, "test_acc":acc_svm, "best_params":gs_svm.best_params_})

# MLP grid
print("\nGridSearch: MLP")
mlp = MLPClassifier(max_iter=300, early_stopping=True, random_state=42)
mlp_param_grid = {
    "hidden_layer_sizes":[(128,), (256,), (256,128)],
    "alpha":[1e-4, 1e-3],
    "learning_rate_init":[1e-3, 5e-4]
}
gs_mlp = GridSearchCV(mlp, mlp_param_grid, cv=3, scoring="accuracy", n_jobs=-1, verbose=1)
gs_mlp.fit(X_train_scaled, y_train)
best_mlp = gs_mlp.best_estimator_
pred_mlp = best_mlp.predict(X_test_scaled)
acc_mlp = accuracy_score(y_test, pred_mlp)
print("Best MLP params:", gs_mlp.best_params_)
print("Best MLP CV accuracy:", gs_mlp.best_score_)
print("Test accuracy (MLP):", acc_mlp)
joblib.dump(gs_mlp, "models/gs_mlp.joblib")
results_grid.append({"model":"MLP", "cv_best":gs_mlp.best_score_, "test_acc":acc_mlp, "best_params":gs_mlp.best_params_})

# Summary
summary_grid = pd.DataFrame(results_grid)
print("\nGrid Search Summary:")
display(summary_grid)


GridSearch: Logistic Regression
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best LR params: {'C': 1.0, 'penalty': 'l2'}
Best LR CV accuracy: 0.7265635170959465
Test accuracy (LR): 0.7128944995491434

GridSearch: SVM (RBF)
Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best SVM params: {'C': 10.0, 'gamma': 0.01}
Best SVM CV accuracy: 0.7459982718335215
Test accuracy (SVM): 0.7397655545536519

GridSearch: MLP
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best MLP params: {'alpha': 0.001, 'hidden_layer_sizes': (256,), 'learning_rate_init': 0.0005}
Best MLP CV accuracy: 0.7506877825422394
Test accuracy (MLP): 0.7439134355275022

Grid Search Summary:


,model,cv_best,test_acc,best_params
0,LogisticRegression,0.726564,0.712894,"{'C': 1.0, 'penalty': 'l2'}"
1,SVM,0.745998,0.739766,"{'C': 10.0, 'gamma': 0.01}"
2,MLP,0.750688,0.743913,"{'alpha': 0.001, 'hidden_layer_sizes': (256,),..."


In [10]:
# ------------------------------------------------------------
# Cell 6 — Optuna tuning (fully fixed version)
# ------------------------------------------------------------
# This may take time. Reduce n_trials if needed.

n_trials = 30
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

def cv_score_estimator(estimator, X_, y_, cv_):
    """Helper to compute mean CV accuracy."""
    return float(np.mean(cross_val_score(estimator, X_, y_, cv=cv_, scoring="accuracy", n_jobs=-1)))


# ============================================================
# 1) Logistic Regression — Optuna Objective
# ============================================================
def objective_logreg(trial):
    C = trial.suggest_loguniform("C", 1e-3, 1e2)

    if n_classes == 2:
        clf = LogisticRegression(
            C=C,
            solver="liblinear",
            multi_class="auto",   # <-- FIXED (cannot be None)
            max_iter=5000,
            random_state=42
        )
    else:
        clf = LogisticRegression(
            C=C,
            solver="lbfgs",
            multi_class="multinomial",
            max_iter=5000,
            random_state=42
        )

    return cv_score_estimator(clf, X_train_scaled, y_train, cv)


# Run study
study_lr = optuna.create_study(direction="maximize", study_name="LR_opt")
t0 = time.time()
study_lr.optimize(objective_logreg, n_trials=n_trials, n_jobs=1)
t1 = time.time()

print(f"\nLogReg Optuna tuning done ({t1 - t0:.1f}s)")
print("Best CV accuracy:", study_lr.best_value)
print("Best LR params:", study_lr.best_params)

# Save study
joblib.dump(study_lr, "optuna_studies/study_lr.pkl")


# Rebuild BEST Logistic Regression model (FIXED)
if n_classes == 2:
    best_lr = LogisticRegression(
        C=study_lr.best_params["C"],
        solver="liblinear",
        multi_class="auto",   # <-- FIXED
        max_iter=5000,
        random_state=42
    )
else:
    best_lr = LogisticRegression(
        C=study_lr.best_params["C"],
        solver="lbfgs",
        multi_class="multinomial",
        max_iter=5000,
        random_state=42
    )

best_lr.fit(X_train_scaled, y_train)
pred_lr_opt = best_lr.predict(X_test_scaled)
test_acc_lr_opt = accuracy_score(y_test, pred_lr_opt)
print("Test accuracy (Optuna LR):", test_acc_lr_opt)


# ============================================================
# 2) SVM — Optuna Objective
# ============================================================
def objective_svm(trial):
    C = trial.suggest_loguniform("C", 1e-2, 1e2)
    gamma_option = trial.suggest_categorical("gamma_option", ["scale", "auto", "float"])

    if gamma_option == "float":
        gamma = trial.suggest_loguniform("gamma_float", 1e-4, 1.0)
    else:
        gamma = gamma_option

    clf = SVC(kernel="rbf", C=C, gamma=gamma, probability=False, random_state=42)
    return cv_score_estimator(clf, X_train_scaled, y_train, cv)


study_svm = optuna.create_study(direction="maximize", study_name="SVM_opt")
t0 = time.time()
study_svm.optimize(objective_svm, n_trials=n_trials, n_jobs=1)
t1 = time.time()

print(f"\nSVM Optuna tuning done ({t1 - t0:.1f}s)")
print("Best CV accuracy:", study_svm.best_value)
print("Best SVM params:", study_svm.best_params)

# Save study
joblib.dump(study_svm, "optuna_studies/study_svm.pkl")

# Reconstruct gamma
svm_params = study_svm.best_params
if svm_params["gamma_option"] == "float":
    gamma_val = svm_params["gamma_float"]
else:
    gamma_val = svm_params["gamma_option"]

best_svm = SVC(
    kernel="rbf",
    C=svm_params["C"],
    gamma=gamma_val,
    probability=False,
    random_state=42
)
best_svm.fit(X_train_scaled, y_train)
pred_svm_opt = best_svm.predict(X_test_scaled)
test_acc_svm_opt = accuracy_score(y_test, pred_svm_opt)
print("Test accuracy (Optuna SVM):", test_acc_svm_opt)


# ============================================================
# 3) MLP — Optuna Objective
# ============================================================
def objective_mlp(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    hidden = []

    for i in range(n_layers):
        hidden.append(trial.suggest_int(f"n_units_l{i}", 64, 512))

    hidden_tuple = tuple(hidden)

    alpha = trial.suggest_loguniform("alpha", 1e-6, 1e-2)
    lr = trial.suggest_loguniform("learning_rate_init", 1e-4, 1e-1)
    batch_size = trial.suggest_categorical("batch_size", [256, 512, 1024])

    clf = MLPClassifier(
        hidden_layer_sizes=hidden_tuple,
        activation="relu",
        solver="adam",
        alpha=alpha,
        learning_rate_init=lr,
        batch_size=batch_size,
        max_iter=200,
        early_stopping=True,
        n_iter_no_change=10,
        validation_fraction=0.1,
        random_state=42
    )
    return cv_score_estimator(clf, X_train_scaled, y_train, cv)


study_mlp = optuna.create_study(direction="maximize", study_name="MLP_opt")
t0 = time.time()
study_mlp.optimize(objective_mlp, n_trials=n_trials, n_jobs=1)
t1 = time.time()

print(f"\nMLP Optuna tuning done ({t1 - t0:.1f}s)")
print("Best CV accuracy:", study_mlp.best_value)
print("Best MLP params:", study_mlp.best_params)

# Save study
joblib.dump(study_mlp, "optuna_studies/study_mlp.pkl")

# Rebuild best MLP
n_layers_best = study_mlp.best_params["n_layers"]
hidden_best = tuple(study_mlp.best_params[f"n_units_l{i}"] for i in range(n_layers_best))

best_mlp = MLPClassifier(
    hidden_layer_sizes=hidden_best,
    activation="relu",
    solver="adam",
    alpha=study_mlp.best_params["alpha"],
    learning_rate_init=study_mlp.best_params["learning_rate_init"],
    batch_size=study_mlp.best_params["batch_size"],
    max_iter=400,
    early_stopping=True,
    n_iter_no_change=10,
    validation_fraction=0.1,
    random_state=42,
    verbose=True
)

best_mlp.fit(X_train_scaled, y_train)
pred_mlp_opt = best_mlp.predict(X_test_scaled)
test_acc_mlp_opt = accuracy_score(y_test, pred_mlp_opt)

print("Test accuracy (Optuna MLP):", test_acc_mlp_opt)


# ============================================================
# Summary Table
# ============================================================
summary_optuna = pd.DataFrame([
    {
        "Model": "Logistic Regression",
        "Best CV Accuracy": study_lr.best_value,
        "Test Accuracy": test_acc_lr_opt,
        "Best Params": study_lr.best_params
    },
    {
        "Model": "SVM (RBF)",
        "Best CV Accuracy": study_svm.best_value,
        "Test Accuracy": test_acc_svm_opt,
        "Best Params": study_svm.best_params
    },
    {
        "Model": "MLP Neural Network",
        "Best CV Accuracy": study_mlp.best_value,
        "Test Accuracy": test_acc_mlp_opt,
        "Best Params": study_mlp.best_params
    }
])

print("\n===== OPTUNA SUMMARY =====")
display(summary_optuna)


[I 2025-12-12 03:56:50,311] A new study created in memory with name: LR_opt
[I 2025-12-12 03:56:50,453] Trial 0 finished with value: 0.7258873236529465 and parameters: {'C': 0.04690088986430441}. Best is trial 0 with value: 0.7258873236529465.
[I 2025-12-12 03:56:50,591] Trial 1 finished with value: 0.7264284186964426 and parameters: {'C': 73.79562689983315}. Best is trial 1 with value: 0.7264284186964426.
[I 2025-12-12 03:56:50,818] Trial 2 finished with value: 0.7264735124915364 and parameters: {'C': 2.583349516596747}. Best is trial 2 with value: 0.7264735124915364.
[I 2025-12-12 03:56:51,023] Trial 3 finished with value: 0.7235875949988962 and parameters: {'C': 0.0017620695901686882}. Best is trial 2 with value: 0.7264735124915364.
[I 2025-12-12 03:56:51,148] Trial 4 finished with value: 0.726248061814645 and parameters: {'C': 0.0712356325895192}. Best is trial 2 with value: 0.7264735124915364.
[I 2025-12-12 03:56:51,384] Trial 5 finished with value: 0.726518612386156 and parameter


LogReg Optuna tuning done (7.0s)
Best CV accuracy: 0.7266989753674794
Best LR params: {'C': 0.1310589505862153}
Test accuracy (Optuna LR): 0.7130748422001804


[I 2025-12-12 03:57:25,827] Trial 0 finished with value: 0.7169591364945078 and parameters: {'C': 0.021944902288771988, 'gamma_option': 'scale'}. Best is trial 0 with value: 0.7169591364945078.
[I 2025-12-12 03:57:52,659] Trial 1 finished with value: 0.7280064148470072 and parameters: {'C': 0.04522508886814044, 'gamma_option': 'float', 'gamma_float': 0.013287656443971398}. Best is trial 1 with value: 0.7280064148470072.
[I 2025-12-12 03:58:18,347] Trial 2 finished with value: 0.7349506458080448 and parameters: {'C': 0.10763974816582872, 'gamma_option': 'auto'}. Best is trial 2 with value: 0.7349506458080448.
[I 2025-12-12 03:58:42,559] Trial 3 finished with value: 0.7394597874359135 and parameters: {'C': 2.5674958516694524, 'gamma_option': 'float', 'gamma_float': 0.0019122145880841207}. Best is trial 3 with value: 0.7394597874359135.
[I 2025-12-12 03:59:08,161] Trial 4 finished with value: 0.7389186679943139 and parameters: {'C': 0.16068598962439093, 'gamma_option': 'scale'}. Best is t


SVM Optuna tuning done (991.4s)
Best CV accuracy: 0.7503718697964944
Best SVM params: {'C': 1.3982188061955807, 'gamma_option': 'auto'}


[I 2025-12-12 04:13:45,731] A new study created in memory with name: MLP_opt


Test accuracy (Optuna SVM): 0.7410279531109107


[I 2025-12-12 04:14:28,944] Trial 0 finished with value: 0.7496503690749937 and parameters: {'n_layers': 3, 'n_units_l0': 220, 'n_units_l1': 444, 'n_units_l2': 289, 'alpha': 4.3683273990846336e-05, 'learning_rate_init': 0.002932373619580776, 'batch_size': 256}. Best is trial 0 with value: 0.7496503690749937.
[I 2025-12-12 04:15:04,736] Trial 1 finished with value: 0.7388281998259342 and parameters: {'n_layers': 2, 'n_units_l0': 483, 'n_units_l1': 459, 'alpha': 0.002813587733993825, 'learning_rate_init': 0.07765807866085371, 'batch_size': 256}. Best is trial 0 with value: 0.7496503690749937.
[I 2025-12-12 04:15:10,556] Trial 2 finished with value: 0.751814724850874 and parameters: {'n_layers': 1, 'n_units_l0': 490, 'alpha': 0.0004614708218905301, 'learning_rate_init': 0.00222737908640218, 'batch_size': 256}. Best is trial 2 with value: 0.751814724850874.
[I 2025-12-12 04:15:15,927] Trial 3 finished with value: 0.7522657359961228 and parameters: {'n_layers': 1, 'n_units_l0': 319, 'alpha'


MLP Optuna tuning done (527.5s)
Best CV accuracy: 0.7522657359961228
Best MLP params: {'n_layers': 1, 'n_units_l0': 319, 'alpha': 7.988148999485019e-05, 'learning_rate_init': 0.0039918337921515295, 'batch_size': 256}
Iteration 1, loss = 0.49914499
Validation score: 0.744815
Iteration 2, loss = 0.47626504
Validation score: 0.750225
Iteration 3, loss = 0.47270401
Validation score: 0.752931
Iteration 4, loss = 0.46832233
Validation score: 0.748873
Iteration 5, loss = 0.46613934
Validation score: 0.751578
Iteration 6, loss = 0.46413286
Validation score: 0.747069
Iteration 7, loss = 0.46374689
Validation score: 0.754734
Iteration 8, loss = 0.46301703
Validation score: 0.753381
Iteration 9, loss = 0.46097405
Validation score: 0.753381
Iteration 10, loss = 0.45819407
Validation score: 0.743463
Iteration 11, loss = 0.45694543
Validation score: 0.754283
Iteration 12, loss = 0.45497905
Validation score: 0.756087
Iteration 13, loss = 0.45418858
Validation score: 0.750225
Iteration 14, loss = 0.4

,Model,Best CV Accuracy,Test Accuracy,Best Params
0,Logistic Regression,0.726699,0.713075,{'C': 0.1310589505862153}
1,SVM (RBF),0.750372,0.741028,"{'C': 1.3982188061955807, 'gamma_option': 'auto'}"
2,MLP Neural Network,0.752266,0.741749,"{'n_layers': 1, 'n_units_l0': 319, 'alpha': 7...."
